In [1]:
from __future__ import generators
from math import *
import random
import sys
import ccmlib
import nef
import nengo
from math import pi
import matplotlib.pyplot as plt
import numpy as np

random.seed(11)

# Parameter declarations 
N = 10
dt = 0.001
ptsc = 0.01

# Define classes useful
class getShoulder():
    def map(self, X):
        x = float(X[0])
        y = float(X[1])
        # make sure we're in the unit circle
        if sqrt(x ** 2 + y ** 2) > 1:
            x = x / (sqrt(x ** 2 + y ** 2))
            y = y / (sqrt(x ** 2 + y ** 2))

        L1 = .5
        L2 = .5
        EPS = 1e-10
        # law of cosines
        D = (x ** 2 + y ** 2 - L1 ** 2 - L2 ** 2) / (2 * L1 * L2)
        if (x ** 2 + y ** 2) < (L1 ** 2 + L2 ** 2):
            D = -D

        # find elbow down angles from shoulder to elbow
        if D < 1 and D > -1:
            elbow = acos(D)
        else:
            elbow = 0

        if (x**2+y**2) < (L1**2+L2**2):
            elbow = pi - elbow

        if x==0 and y==0:
            y = y+EPS

        inside = L2 * sin(elbow) / (sqrt(x ** 2 + y ** 2))
        if inside > 1:
            inside = 1
        if inside < -1:
            inside = -1

        if x==0:
            shoulder = 1.5708 - asin(inside)
        else:
            shoulder = atan(y/x) - asin(inside)
        if x < 0:
            shoulder = shoulder + pi

        return shoulder

    def getDimension(self):
        return 2
    
    
class getElbow():
    def map(self, X):
        x = float(X[0])
        y = float(X[1])
        # make sure we're in the unit circle
        if sqrt(x ** 2 + y ** 2) > 1:
            x = x / (sqrt(x ** 2 + y ** 2))
            y = y / (sqrt(x ** 2 + y ** 2))
        L1 = .5
        L2 = .5
        # law of cosines
        D = (x ** 2 + y ** 2 - L1 ** 2 - L2 ** 2) / (2 * L1 * L2)
        if (x ** 2 + y ** 2) < (L1 ** 2+ L2 ** 2):
            D = -D

        # find elbow down angles from shoulder to elbow
        if D < 1 and D > -1:
            elbow = acos(D)
        else:
            elbow = 0

        if (x ** 2 + y ** 2) < (L1 ** 2 + L2 ** 2):
            elbow = pi - elbow

        return elbow

    def getDimension(self):
        return 2


class getX():
    def map(self, X):
        shoulder = X[0]
        elbow = X[1]
        L1 = .5
        L2 = .5

        return L1*cos(shoulder)+L2*cos(shoulder+elbow)

    def getDimension(self):
        return 2

class getY():
    def map(self, X):
        shoulder = X[0]
        elbow = X[1]
        L1 = .5
        L2 = .5
        
        return L1 * sin(shoulder) + L2 * sin(shoulder + elbow)

    def getDimension(self):
        return 2




# Neural part
model = nengo.Network(label="Big Basic")

with model:
    # input functions
    refX = nengo.Node(output=1,label = "Xref")
    refY = nengo.Node(output=1,label = "Yref") 
    Tfunc = nengo.Node(output = [1, 0, 0, 1], label = "Tmatrix")
    F = nengo.Node(output = [-1, 0, -1, 0, 0, -1, 0, -1], label = "F")

    # neural populations
    convertXY = nengo.Ensemble(N, dimensions = 2,label=="convert XY")
    convertAngles = nengo.Ensemble(N, dimensions = 2,label="convert angles")
    funcT = nengo.Ensemble(N, dimensions = 6,label="Funcion T")
    FX = nengo.Ensemble(N, dimensions = 12,label="FX")
    # calculate 2D control signal
    controlV = nengo.Ensemble(N, dimensions = 2,label="control signal v")
    # calculates joint torque control signal
    controlU = nengo.Ensemble(
        N,dimensions = 2,label="control signal u")
    
    # Decoding terminations
    # Passthrough nodes
    refXY = nengo.Node(size_in = 2, label = "refXY")
    refXYElbow = nengo.Node(size_in =2, label = "elbowRef")
    refXYShoulder = nengo.Node(size_in =2, label = "shoulderRef")
    nengo.Connection(convertXY,refXY,transform = [[1,0],[0,1]])
    
    refShoulderAngles = nengo.Node(size_in = 1, label = "ShoulderAngle")
    refElbowAngles = nengo.Node(size_in = 1, label = "ElbowAngle")
    nengo.Connection(refShoulderAngles,convertAngles,transform = [[1],[0]])
    nengo.Connection(refElbowAngles,convertAngles,transform = [[0],[1]])
    

